In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, time, timedelta
import json

In [3]:
security_data = pd.read_csv('data\security_logs.csv')

In [84]:
class Visit(dict):
    def __init__(self, student_id, time_start, time_end, location):
        self._student_id = student_id;
        self._location = location;
        self._start = str(time_start);
        self._end = str(time_end);
        dict.__init__(self, student_id=self._student_id, location=self._location, start=self._start, end=self._end)
    
    @property
    def student_id(self):
        return self._student_id
    
    @property
    def location(self):
        return self._location;
    
    def __str__(self):
        return self._student_id + " " + self._location
    
    def __repr__(self):
        return self._location + " - " + self._student_id + " : "  + str(self._start) + " - " + str(self._end)

In [138]:
class Lecture(dict):
    def __init__(self, lecture_name, start, end):
        self._name = lecture_name
        self._start = str(start)
        self._end = str(end)
        dict.__init__(self, name=self._name, start=self._start, end=self._end)
        
    def __str__(self):
        return self._name + " : " + self._start + " - " + self._end 
    
    def __repr__(self):
        return self._name + " : " + self._start + " - " + self._end 

In [216]:
class Person(dict):
    def __init__(self, name, student_id):
        self._name = name;
        self._student_id = student_id;
        self._visits = []
        self._interview = ""
        dict.__init__(self, name=self._name, student_id=self._student_id, visits=self._visits, interview=self._interview)
        
    @property
    def name(self):
        return self._name
    
    @property
    def visits(self):
        return self._visits
    
    @property
    def student_id(self):
        return self._student_id
    
    def add_visit(self, visit):
        self._visits.append(visit)
        dict.__init__(self, name=self._name, student_id=self._student_id, visits=self._visits, interview=self._interview)
        
    def add_interview(self, interview):
        self._interview = interview
    
    def __str__(self):
        return self._student_id + " : " + self._name 

    def __repr__(self):
        return self._student_id + " : " + self._name
        

In [217]:
class Building(dict):
    def __init__(self, building_name, open_time, close_time, geo_loc, description):
        self._name = building_name
        self._opens = str(open_time)
        self._closes = str(close_time)
        self._geo_loc = geo_loc
        self._description = description
        self._visits = []
        dict.__init__(self, name=self._name, opens=self._opens, closes=self._closes, 
                    geo_loc=self._geo_loc, description=self._description,
                    visits=self._visits)
    @property
    def name(self):
        return self._name
    
    @property
    def description(self):
        return self._description
    
    def add_visit(self, visit):
        self._visits.append(visit)
        dict.__init__(self, name=self._name, opens=self._opens, closes=self._closes, 
                    geo_loc=self._geo_loc, description=self._description,
                    visits=self._visits)
    @property
    def lecture(self):
        return self._lecture
    
    def add_lecture(self, lecture):
        self._lecture = lecture
    
    def __str__(self):
        return self._name + " : " + str(self._opens) + " - " + str(self._closes)
    
    def __repr__(self):
        return self._name + " - " + str(self._lecture) + " : " + str(self._opens) + " - " + str(self._closes)

In [218]:
def convert_to_datetime(time):
    time = time.strip()
    time_hour = int(time[0:2])
    time_minutes = int(time[2:4])
    day = 11 if time_hour < 6 else 10
    return datetime(2020, 10, day, time_hour, time_minutes)

In [219]:
timedataframe = pd.DataFrame(security_data.Time.value_counts())
time  = security_data['Time'].str.split('-', expand = True)
security_df = pd.concat([security_data, time], axis = 1)
security_df = security_df.drop('Time', axis = 1)
security_df = security_df.rename(columns = {0: 'Start', 1: 'End'})
security_df['Start'] = security_df['Start'].apply(convert_to_datetime)
security_df['End'] = security_df['End'].apply(convert_to_datetime)

In [220]:
people_file = open("data\people_data.csv")
people_lines = people_file.readlines()
people_file.close()
all_people = []
for line in people_lines[1:]:
    split_line = line.strip().split(",")
    student_id = split_line[0]
    name = split_line[1]
    person = Person(name, student_id)
    all_people.append(person)

In [237]:
statements_file = open("Statements.csv")
statements_lines = statements_file.readlines()
statements_file.close()
for line in statements_lines:
    split_line = line.strip().split(",")
    name = split_line[0]
    interview = split_line[1].strip()
    for person in all_people:
        if name == person.name:
            person.add_interview(interview)

In [232]:
locations_file = open("data\location_data.csv")
locations_lines = locations_file.readlines()
locations_file.close()
all_locations = []
for line in locations_lines[1:]:
    split_line = line.strip().split(",")
    name = split_line[0]
    geo_loc = list(map(float, split_line[1][1:-1].split()))
    opens, closes = list(map(convert_to_datetime, split_line[2].split("-")))
    description = split_line[3]
    location = Building(name, opens, closes, geo_loc, description)
    all_locations.append(location)

[55.8730105, -4.2925361]
[55.8708777, -4.2893462]
[55.8737694, -4.2920551]
[55.8714732, -4.2927198]
[55.8729239, -4.295317]
[55.8723758, -4.285949]
[55.8728276, -4.2844698]
[55.8739511, -4.2940459]
[55.8739571, -4.2940459]
[55.8735992, -4.2935842]
[55.8717345, -4.2818099]
[55.8689911, -4.2847528]
[55.8721791, -4.2925265]
[55.8721791, -4.2925265]


In [233]:
lecture_file = open("data\lecture_data.csv")
lecture_lines = lecture_file.readlines()
lecture_file.close()
all_lectures = []
for line in lecture_lines[1:]:
    split_line = line.strip().split(",")
    building_name = split_line[0]
    opens, closes = list(map(convert_to_datetime, split_line[1].split("-")))
    lecture_name = split_line[2]
    lecture_start, lecture_end = list(map(convert_to_datetime, split_line[1].split("-")))
    lecture = Lecture(lecture_name, lecture_start, lecture_end)
    for location in all_locations:
        if location.name == building_name:
            location.add_lecture(lecture)
            break

In [234]:
visits_file = open("data\security_logs.csv")
visits_lines = visits_file.readlines()
visits_file.close()
for line in visits_lines[1:]:
    split_line = line.strip().split(",")
    student_id = split_line[0]
    building_name = split_line[2]
    start_time, end_time = list(map(convert_to_datetime, split_line[3].split("-")))
    visit = Visit(student_id, start_time, end_time, building_name)
    for person in all_people:
        if person.student_id == student_id:
            person.add_visit(visit)
            break
    for location in all_locations:
        if location.name == building_name:
            location.add_visit(visit)
            break

In [238]:
with open('data\people_json.json', 'w') as f:
    json.dump(all_people, f)

In [236]:
with open("data\location_json.json", 'w') as f:
    json.dump(all_locations, f)